![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_XlmRoBERTaForMultipleChoice.ipynb)

## Import ONNX XlmRoBERTaForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `XlmRoBertaForMultipleChoice` is only available since in `Spark NLP 5.6.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for question answering via `XlmRoBertaForMultipleChoice` or `TFXlmRobertaForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [XlmRoBertaForMultipleChoice](https://huggingface.co/docs/transformers/en/model_doc/xlm-roberta#transformers.XLMRobertaModel)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [lenatr99/fine_tuned_copa_XLMroberta](https://huggingface.co/lenatr99/fine_tuned_copa_XLMroberta) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLMRobertaTokenizer
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "lenatr99/fine_tuned_copa_XLMroberta"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

('onnx_models/lenatr99/fine_tuned_copa_XLMroberta/tokenizer_config.json',
 'onnx_models/lenatr99/fine_tuned_copa_XLMroberta/special_tokens_map.json',
 'onnx_models/lenatr99/fine_tuned_copa_XLMroberta/sentencepiece.bpe.model',
 'onnx_models/lenatr99/fine_tuned_copa_XLMroberta/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -lh {EXPORT_PATH}

total 1.1G
-rw-r--r-- 1 root root  733 Jun 26 15:26 config.json
-rw-r--r-- 1 root root 1.1G Jun 26 15:26 model.onnx
-rw-r--r-- 1 root root 4.9M Jun 26 15:26 sentencepiece.bpe.model
-rw-r--r-- 1 root root  964 Jun 26 15:26 special_tokens_map.json
-rw-r--r-- 1 root root 1.2K Jun 26 15:26 tokenizer_config.json


We are missing the `sentencepiece.bpe.model` from the tokenizer. Let's move this file to `assets` folder which Spark NLP will look for.

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [5]:
!ls -lR {EXPORT_PATH}

onnx_models/lenatr99/fine_tuned_copa_XLMroberta:
total 1086448
drwxr-xr-x 2 root root       4096 Jun 26 15:26 assets
-rw-r--r-- 1 root root        733 Jun 26 15:26 config.json
-rw-r--r-- 1 root root 1112499868 Jun 26 15:26 model.onnx
-rw-r--r-- 1 root root        964 Jun 26 15:26 special_tokens_map.json
-rw-r--r-- 1 root root       1202 Jun 26 15:26 tokenizer_config.json

onnx_models/lenatr99/fine_tuned_copa_XLMroberta/assets:
total 4952
-rw-r--r-- 1 root root 5069051 Jun 26 15:26 sentencepiece.bpe.model


Voila! We have our `sentencepiece.bpe.model` inside assets directory

## Import and Save XlmRoBertaForMultipleChoice in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q spark-nlp==6.0.3 pyspark==3.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 43.5 MB/s eta 0:00:00


In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  6.0.3
Apache Spark version:  3.5.1


- Let's use `loadSavedModel` functon in `XlmRoBertaForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import XlmRoBertaForMultipleChoice

xlmRoBertaMultpleChoiceClassifier = XlmRoBertaForMultipleChoice.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [9]:
xlmRoBertaMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `XlmRoBertaForMultipleChoice` model in Spark NLP 🚀 pipeline!

In [11]:
data = spark.createDataFrame([
    ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
     "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against invasions from which group?",
     "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygenm, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
], ["question", "choices"])

data.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

In [12]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import XlmRoBertaForMultipleChoice
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "choices"]) \
    .setOutputCols(["document_question", "document_choices"])

xlm_roberta_model = XlmRoBertaForMultipleChoice() \
    .load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_choices"]) \
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    xlm_roberta_model
])

model = pipeline.fit(data)
results = model.transform(data)

results.selectExpr(
    "question",
    "choices",
    "answer.result[0] as predicted_answer"
).show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|question                                                                                  |choices                                                                 |predicted_answer                   |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|It is eaten with a fork and a knife|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |Germany                    